In [1]:
from typing import Any
from langchain import LLMChain

import openai
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain


In [2]:
"""Chain of Density prompting: https://arxiv.org/abs/2309.04269"""
user_sum_prompt = """
Article: {text}
You will generate increasingly concise entity-dense summaries of the above article. Repeat the following 2 steps 5 times.

Step 1: Identify 1-3 informative entities (delimited) from the article which are missing from the previously generated summary.
Step 2: Write a new denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.

A missing entity is
- Relevant: to the main stories.
- Specific: descriptive yet concise (5 words or fewer).
- Novel: not in the previous summary.
- Faithful: present in the article.
- Anywhere: located in the article.

Guidelines:
- The first summary should be long (4-5 sentences, ~80 words), yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
- Make every word count. Rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise, yet self-contained, e.g., easily understood without the article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

Remember: Use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "missing_entities" and "denser_summary".
"""

user_translate_prompt = """
Text: {sum}

Übersetze den Text in {language}. Beinhalte die Formatierung als JSON bei.
"""


In [3]:
def getSummarizationPrompt() -> PromptTemplate:
    return PromptTemplate(input_variables=["text"], template=user_sum_prompt)

def getTranslationPrompt() -> PromptTemplate:
    return PromptTemplate(input_variables=["language", "sum"], template=user_translate_prompt)


In [4]:
import os
os.environ['http_proxy'] = "http://internet-proxy-client.muenchen.de:80" 
os.environ['https_proxy'] = "http://internet-proxy-client.muenchen.de:80" 

In [5]:
openai.api_type = "azure"
openai.api_base = f"https://cog-tb726faeoukc2.openai.azure.com"
openai.api_version = "2023-05-15"
openai.api_key = "9324d0a0d0bd4d72b0e34596a9dc4a9c"

In [41]:
from typing import Dict
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema.output import LLMResult
class MyCustomHandler(BaseCallbackHandler):
    def on_text(self, text: str, **kwargs: Any) -> Any:
        print(f"Text: {text}")
        self.log = text

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        print(response.llm_output.keys())
        print(response.llm_output)

In [42]:
verbose = True
llm = AzureChatOpenAI(
    model="gpt-35-turbo",
    temperature= 0.7,
    max_tokens=4096,
    n=1,
    deployment_name= "chat",
    openai_api_key=openai.api_key,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    openai_api_type=openai.api_type,
    callbacks=[MyCustomHandler()]
)
summarizationChain = LLMChain(llm=llm, prompt=getSummarizationPrompt(), output_key="sum", verbose=verbose)
translationChain = LLMChain(llm=llm, prompt=getTranslationPrompt(), output_key="translation", verbose=verbose)
overall_chain = SequentialChain(
    chains=[summarizationChain, translationChain], 
    input_variables=["language", "text"],
    output_variables=["translation", "sum"],
    verbose=verbose
)

In [43]:
text ="""Der Zoo Hof vermisst seit dem Wochenende das Känguru Willi. Das Tier sei seitdem mehrmals gesichtet worden, sagte ein Polizeisprecher am Dienstag. "Es ist bisher nicht gelungen, es einzufangen." Mehrere Medien hatten zuvor darüber berichtet.

Der Zoo schrieb auf Instagram, dass das graubraune Känguru-Männchen Willi am Samstagmorgen nicht mehr im Gehege gewesen sei. Wie das etwa 70 Zentimeter große Tier den Zoo verlassen konnte, sei noch in Klärung. "Da das Tier von Blättern und Gras lebt, befindet es sich momentan nicht in der Gefahr des Verhungerns, einzig die Beutegreifer in der Natur könnten ihm gefährlich werden. Wer das Känguru sehe, soll es nur aus der Ferne beobachten und die Polizei verständigen, bat der Zoo im bayerischen Oberfranken. "Bitte unterlassen Sie Suchen auf eigene Faust, dies würde das Tier nur noch weiter unnötig stressen."

Entlaufene Kängurus wurden in den vergangenen Jahren immer mal wieder in Bayern gemeldet. Zuletzt war im Sommer das Känguru-Männchen Toni aus dem Gehege eines Hotels im Bayerischen Wald ausgebüxt. Mit einer Spur aus Leckereien konnten die Besitzer es nach mehreren Wochen schließlich in sein Gehege zurück locken."""
text ="""Autonome Autos sind Autos, die komplett selbstständig fahren. Sie brauchen keinen Fahrer mehr. Ihr Innenraum könnte beispielsweise wie in einem Zug aussehen, mit vier Sitzen und eventuell einem Tisch. Lenkrad, Pedale, Schaltknüppel – all das wäre nicht mehr vorhanden. Damit diese Autos autonom fahren können, sind sie mit einer Fülle von Technik ausgestattet. Laser, Sensoren, Kameras, ein GPS-Empfänger, Messgeräte und ein Bordcomputer erkennen andere Verkehrsteilnehmer, Straßenschilder, Ampeln usw. Sie machen es möglich, dass das Auto sich sicher durch den Verkehr bewegt und Kollisionen vermeidet. Mehrere Autohersteller arbeiten an verschiedenen Modellen autonomer Autos und die Forschung ist schon weit fortgeschritten."""
language="deutsch"

In [46]:

from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    result = overall_chain({"text": text, "language": language})
total_tokens = cb.total_tokens



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Article: Autonome Autos sind Autos, die komplett selbstständig fahren. Sie brauchen keinen Fahrer mehr. Ihr Innenraum könnte beispielsweise wie in einem Zug aussehen, mit vier Sitzen und eventuell einem Tisch. Lenkrad, Pedale, Schaltknüppel – all das wäre nicht mehr vorhanden. Damit diese Autos autonom fahren können, sind sie mit einer Fülle von Technik ausgestattet. Laser, Sensoren, Kameras, ein GPS-Empfänger, Messgeräte und ein Bordcomputer erkennen andere Verkehrsteilnehmer, Straßenschilder, Ampeln usw. Sie machen es möglich, dass das Auto sich sicher durch den Verkehr bewegt und Kollisionen vermeidet. Mehrere Autohersteller arbeiten an verschiedenen Modellen autonomer Autos und die Forschung ist schon weit fortgeschritten.
You will generate increasingly concise entity-dense summaries of the above article. Repeat the following 2 steps 5 times.

Step 1: Identify 1-3 informative enti

In [47]:
result, total_tokens

({'text': 'Autonome Autos sind Autos, die komplett selbstständig fahren. Sie brauchen keinen Fahrer mehr. Ihr Innenraum könnte beispielsweise wie in einem Zug aussehen, mit vier Sitzen und eventuell einem Tisch. Lenkrad, Pedale, Schaltknüppel – all das wäre nicht mehr vorhanden. Damit diese Autos autonom fahren können, sind sie mit einer Fülle von Technik ausgestattet. Laser, Sensoren, Kameras, ein GPS-Empfänger, Messgeräte und ein Bordcomputer erkennen andere Verkehrsteilnehmer, Straßenschilder, Ampeln usw. Sie machen es möglich, dass das Auto sich sicher durch den Verkehr bewegt und Kollisionen vermeidet. Mehrere Autohersteller arbeiten an verschiedenen Modellen autonomer Autos und die Forschung ist schon weit fortgeschritten.',
  'language': 'deutsch',
  'translation': '[{"missing_entities": "Autonome Autos", "denser_summary": "Autonome Autos fahren komplett selbstständig, ohne Fahrer. Sie sind mit Technik wie Laser, Sensoren, Kameras, GPS-Empfänger, Messgeräten und Bordcomputern au

In [48]:
cb

Tokens Used: 2856
	Prompt Tokens: 1326
	Completion Tokens: 1530
Successful Requests: 2
Total Cost (USD): $0.005712

In [122]:
result["translation"].replace("\n", "")

'[  {    "missing_entities": "GPS-Empfänger",    "denser_summary": "Autonome Autos nutzen Laser, Sensoren, Kameras, GPS-Empfänger, Messgeräte und einen Bordcomputer für eine sichere Navigation. Sie benötigen keinen Fahrer und haben kein Lenkrad, keine Pedale und keinen Gangschalter. Mehrere Autohersteller entwickeln verschiedene autonome Automodelle."  },  {    "missing_entities": "Verkehrsteilnehmer",    "denser_summary": "Autonome Autos mit eingebautem Laser, Sensoren, Kameras, GPS-Empfänger, Messgeräten und Bordcomputer vermeiden Unfälle, indem sie andere Fahrzeuge, Verkehrsschilder und Ampeln erkennen. Sie benötigen keinen Fahrer und haben kein Lenkrad, keine Pedale und keinen Gangschalter. Mehrere Autohersteller entwickeln verschiedene autonome Automodelle."  },  {    "missing_entities": "Forschung",    "denser_summary": "Mehrere Autohersteller entwickeln autonome Autos mit Laser, Sensoren, Kameras, GPS-Empfänger, Messgeräten und einem Bordcomputer. Sie benötigen keinen Fahrer und

In [123]:
import json

In [124]:
jsoned = json.loads(result["translation"])

In [129]:
cleaned = []
for (i, element) in enumerate(jsoned):
    print(element)
    missing = element['missing_entities']
    if(isinstance(missing, str)):
        element['missing_entities'] = [missing]
    cleaned.append(element)

{'missing_entities': 'GPS-Empfänger', 'denser_summary': 'Autonome Autos nutzen Laser, Sensoren, Kameras, GPS-Empfänger, Messgeräte und einen Bordcomputer für eine sichere Navigation. Sie benötigen keinen Fahrer und haben kein Lenkrad, keine Pedale und keinen Gangschalter. Mehrere Autohersteller entwickeln verschiedene autonome Automodelle.'}
GPS-Empfänger
{'missing_entities': 'Verkehrsteilnehmer', 'denser_summary': 'Autonome Autos mit eingebautem Laser, Sensoren, Kameras, GPS-Empfänger, Messgeräten und Bordcomputer vermeiden Unfälle, indem sie andere Fahrzeuge, Verkehrsschilder und Ampeln erkennen. Sie benötigen keinen Fahrer und haben kein Lenkrad, keine Pedale und keinen Gangschalter. Mehrere Autohersteller entwickeln verschiedene autonome Automodelle.'}
Verkehrsteilnehmer
{'missing_entities': 'Forschung', 'denser_summary': 'Mehrere Autohersteller entwickeln autonome Autos mit Laser, Sensoren, Kameras, GPS-Empfänger, Messgeräten und einem Bordcomputer. Sie benötigen keinen Fahrer und

In [130]:
cleaned

[{'missing_entities': ['GPS-Empfänger'],
  'denser_summary': 'Autonome Autos nutzen Laser, Sensoren, Kameras, GPS-Empfänger, Messgeräte und einen Bordcomputer für eine sichere Navigation. Sie benötigen keinen Fahrer und haben kein Lenkrad, keine Pedale und keinen Gangschalter. Mehrere Autohersteller entwickeln verschiedene autonome Automodelle.'},
 {'missing_entities': ['Verkehrsteilnehmer'],
  'denser_summary': 'Autonome Autos mit eingebautem Laser, Sensoren, Kameras, GPS-Empfänger, Messgeräten und Bordcomputer vermeiden Unfälle, indem sie andere Fahrzeuge, Verkehrsschilder und Ampeln erkennen. Sie benötigen keinen Fahrer und haben kein Lenkrad, keine Pedale und keinen Gangschalter. Mehrere Autohersteller entwickeln verschiedene autonome Automodelle.'},
 {'missing_entities': ['Forschung'],
  'denser_summary': 'Mehrere Autohersteller entwickeln autonome Autos mit Laser, Sensoren, Kameras, GPS-Empfänger, Messgeräten und einem Bordcomputer. Sie benötigen keinen Fahrer und haben kein Lenk